# Cleaning the data for KNN+Regression for Question 3

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import matplotlib.path as mplPath
#import rtree
#import fiona.crs
#import geopandas as gpd
#import pyproj
#import shapely.geometry as geom
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [6]:
january2013=pd.read_csv("datasets/yellow_tripdata_2013-01.csv")

In [7]:
january2013 = january2013.loc[(january2013['pickup_longitude'] > -74.06) & (january2013['pickup_longitude'] < -73.77) & (january2013['pickup_latitude'] > 40.61) &  (january2013['pickup_latitude'] < 40.91)]
#january2013 = january2013.loc[(january2013['passenger_count'] > 0) & (january2013['passenger_count'] < 7)]
january2013 = january2013.reset_index()
january2013 = january2013.drop('index', 1)
january2013 = january2013.drop('vendor_id',1)
january2013 = january2013.drop('rate_code',1)
january2013 = january2013.drop('store_and_fwd_flag',1)
january2013 = january2013.drop('fare_amount',1)
january2013 = january2013.drop('surcharge',1)
january2013 = january2013.drop('mta_tax',1)
january2013 = january2013.drop('tolls_amount',1)
january2013 = january2013.drop('dropoff_latitude',1)
january2013 = january2013.drop('dropoff_longitude',1)
january2013 = january2013.drop('passenger_count',1)
january2013 = january2013.drop('payment_type',1)

In [8]:
def add_data(df):
    df_timestamp = pd.to_datetime(pd.Series(df['pickup_datetime']))
    df['trip_distance']*0.621371 # convert to miles
    df['weekday'] = df_timestamp.dt.weekday_name
    #df['month'] = df_timestamp.dt.month
    df['hour'] = df_timestamp.dt.hour
    #df['day'] = df_timestamp.dt.day
    #df['minutes'] = (df_timestamp.dt.hour)*60 + df_timestamp.dt.minute
    time_spent = pd.to_datetime(df['dropoff_datetime']) - pd.to_datetime(df['pickup_datetime'])
    df['time_spent'] = pd.to_datetime(time_spent).dt.minute
    df['pickup'] = df['pickup_latitude'].map(str) +','+df['pickup_longitude'].map(str)
    return df

In [9]:
january2013 = add_data(january2013)

In [10]:
january2013 = january2013.drop('pickup_datetime',1)
january2013 = january2013.drop('dropoff_datetime',1)
january2013 = january2013.drop('pickup_longitude',1)
january2013 = january2013.drop('pickup_latitude',1)

In [11]:
january2013.head()

,trip_distance,tip_amount,total_amount,weekday,hour,time_spent,pickup
0,1.0,0.0,7.0,Tuesday,15,6,"40.757977,-73.978165"
1,1.5,0.0,7.0,Sunday,0,4,"40.731781,-74.00668"
2,1.1,0.0,7.0,Saturday,18,4,"40.73777,-74.004711"
3,0.7,0.0,6.0,Monday,23,4,"40.759945,-73.97459999999998"
4,2.1,0.0,10.5,Monday,23,9,"40.748528,-73.976252"


In [12]:
# Look into the dataframe 
# First by weekday, then hour then block
# and we will know the average tip amount
# for each weekday > hour > block
# eg. On (day of week) at (hour) on (lat,long) avg tip is $number
def get_avg_tips(df):
    avg_tips = df.groupby(['weekday','hour','pickup']).mean()
    avg_tips = avg_tips.reset_index()
    return avg_tips

In [13]:
january2013 = get_avg_tips(january2013)

In [14]:
january2013.shape

(14256773, 7)

In [15]:
january2013.head()

,weekday,hour,pickup,trip_distance,tip_amount,total_amount,time_spent
0,Friday,0,"40.613763,-73.972592",0.00,7.700,38.500,0.0
1,Friday,0,"40.615297,-73.96423699999998",12.22,0.000,36.500,27.0
2,Friday,0,"40.61622,-73.97454999999998",3.70,1.625,16.375,13.0
3,Friday,0,"40.616755,-73.944947",1.05,0.000,6.000,3.0
4,Friday,0,"40.617259,-73.96366999999998",1.40,0.000,7.500,4.0


In [16]:
january2013.tail()

,weekday,hour,pickup,trip_distance,tip_amount,total_amount,time_spent
14256768,Wednesday,23,"40.890857,-73.908495",0.0,4.41,7.91,1.0
14256769,Wednesday,23,"40.892463,-73.816218",13.6,0.00,3.50,0.0
14256770,Wednesday,23,"40.900912,-74.00320499999998",0.0,23.00,75.50,0.0
14256771,Wednesday,23,"40.902505,-74.00228799999998",0.0,3.00,86.00,6.0
14256772,Wednesday,23,"40.907027,-73.909115",0.0,9.50,50.50,0.0


In [17]:
january2013.to_csv('clean-january-2013.csv',sep=',',encoding='utf-8')

Do not run the above and below parts together... I'll split this into 2 notebooks later.

# Now we will try K Nearest Neighbors with Regression on the cleaned data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import matplotlib.path as mplPath
#import rtree
#import fiona.crs
#import geopandas as gpd
#import pyproj
#import shapely.geometry as geom
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
january2013=pd.read_csv("datasets/clean-january-2013.csv")

In [3]:
january2013.head()

,Unnamed: 0,weekday,hour,pickup,trip_distance,tip_amount,total_amount,time_spent
0,0,Friday,0,"40.613763,-73.972592",0.00,7.700,38.500,0.0
1,1,Friday,0,"40.615297,-73.96423699999998",12.22,0.000,36.500,27.0
2,2,Friday,0,"40.61622,-73.97454999999998",3.70,1.625,16.375,13.0
3,3,Friday,0,"40.616755,-73.944947",1.05,0.000,6.000,3.0
4,4,Friday,0,"40.617259,-73.96366999999998",1.40,0.000,7.500,4.0


In [4]:
january2013.tail()

,Unnamed: 0,weekday,hour,pickup,trip_distance,tip_amount,total_amount,time_spent
14256768,14256768,Wednesday,23,"40.890857,-73.908495",0.0,4.41,7.91,1.0
14256769,14256769,Wednesday,23,"40.892463,-73.816218",13.6,0.00,3.50,0.0
14256770,14256770,Wednesday,23,"40.900912,-74.00320499999998",0.0,23.00,75.50,0.0
14256771,14256771,Wednesday,23,"40.902505,-74.00228799999998",0.0,3.00,86.00,6.0
14256772,14256772,Wednesday,23,"40.907027,-73.909115",0.0,9.50,50.50,0.0


In [5]:
# Create indicator variables for the hours and days of the week
#and drop the categorical values
Xnames = ['weekday','hour','pickup','trip_distance',
          'tip_amount','total_amount','time_spent']
X = january2013[Xnames]
X = X.join(pd.get_dummies(X['hour']))
X = X.join(pd.get_dummies(X['weekday']))
X = X.drop(['hour','weekday'], axis=1)

# Split off the target (which will be the logarithm of the number of pickups (+1))
y = january2013['tip_amount']

In [10]:
X.head()

,pickup,trip_distance,tip_amount,total_amount,time_spent,0,1,2,3,4,...,21,22,23,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,"40.613763,-73.972592",0.00,7.700,38.500,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"40.615297,-73.96423699999998",12.22,0.000,36.500,27.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"40.61622,-73.97454999999998",3.70,1.625,16.375,13.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"40.616755,-73.944947",1.05,0.000,6.000,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"40.617259,-73.96366999999998",1.40,0.000,7.500,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
itrain, itest = train_test_split(range(january2013.shape[0]), train_size=0.8)
mask=np.ones(january2013.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [6]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]
Xtrain.head()

,pickup,trip_distance,tip_amount,total_amount,time_spent,0,Friday
0,"40.613763,-73.972592",0.00,7.700,38.500,0.0,1.0,1.0
1,"40.615297,-73.96423699999998",12.22,0.000,36.500,27.0,1.0,1.0
2,"40.61622,-73.97454999999998",3.70,1.625,16.375,13.0,1.0,1.0
3,"40.616755,-73.944947",1.05,0.000,6.000,3.0,1.0,1.0
4,"40.617259,-73.96366999999998",1.40,0.000,7.500,4.0,1.0,1.0


In [8]:
Xtrain_normalized = (Xtrain - Xtrain.mean())/Xtrain.std()
Xtest_normalized = (Xtest - Xtrain.mean())/Xtrain.std()

C:\Users\Victor\Anaconda3\lib\site-packages\pandas\core\generic.py:4267: RuntimeWarning: unorderable types: str() > int(), sort order is undefined for incomparable objects
  return_indexers=True)
C:\Users\Victor\Anaconda3\lib\site-packages\pandas\core\ops.py:995: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 8000
  yrav = yrav[mask]
C:\Users\Victor\Anaconda3\lib\site-packages\pandas\core\ops.py:995: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 2000
  yrav = yrav[mask]


In [28]:
Xtest_normalized = Xtest_normalized.drop('pickup',1)
Xtest_normalized = Xtest_normalized.fillna(0)
Xtest_normalized.head()

,trip_distance,tip_amount,total_amount,time_spent,0,Friday
15,0.304922,0.808577,0.366407,0.865675,0.0,0.0
23,-0.849380,-0.687389,-0.923583,-1.332985,0.0,0.0
30,2.805910,1.012572,2.602791,3.430779,0.0,0.0
31,2.407676,2.576537,2.277279,1.476414,0.0,0.0
38,0.997504,-0.687389,0.703974,0.987823,0.0,0.0


In [29]:
Xtrain_normalized = Xtrain_normalized.drop('pickup',1)
Xtrain_normalized = Xtrain_normalized.fillna(0)
Xtrain_normalized.head()

ValueError: labels ['pickup'] not contained in axis

In [30]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print("BEST", gs.best_params_, gs.best_score_, gs.grid_scores_)
    best = gs.best_estimator_
    return best

In [31]:
%%time

# Create a k-Nearest Neighbors Regression estimator
knn_estimator = KNeighborsRegressor()

# Define a grid of parameters over which to optimize the knn regressor
# We will figure out which number of neighbors is optimal
#knn_parameters = {"n_neighbors": [1,2,5,10,20,50,100]}

knn_parameters = {"n_neighbors": [1,2,5]}
knn_best = cv_optimize(knn_estimator, knn_parameters, Xtrain_normalized, ytrain, score_func='mean_squared_error')

BEST {'n_neighbors': 2} -0.181913696776 [mean: -0.19285, std: 0.31578, params: {'n_neighbors': 1}, mean: -0.18191, std: 0.29833, params: {'n_neighbors': 2}, mean: -0.19547, std: 0.32173, params: {'n_neighbors': 5}]
Wall time: 309 ms


In [32]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
knn_reg=knn_best.fit(Xtrain_normalized, ytrain)
knn_training_accuracy = knn_reg.score(Xtrain_normalized, ytrain)
knn_test_accuracy = knn_reg.score(Xtest_normalized, ytest)
print("R^2 on training data: %0.4f" % (knn_training_accuracy))
print("R^2 on test data:     %0.4f" % (knn_test_accuracy))

R^2 on training data: 0.9972
R^2 on test data:     0.9717


In [35]:
np.sqrt(mean_squared_error(knn_reg.predict(Xtest_normalized),ytest))

0.54502334635406913

In [48]:
# Show some of the predictions vs. the real number of pickups
# predictions vs. real number of pickups
np.column_stack((knn_reg.predict(Xtest_normalized),ytest))

array([[ 4.35,  4.4 ],
       [ 0.  ,  0.  ],
       [ 5.  ,  5.  ],
       ..., 
       [ 0.  ,  0.  ],
       [ 0.  ,  0.  ],
       [ 0.  ,  0.  ]])

In [52]:
Xtest_normalized.head()

,trip_distance,tip_amount,total_amount,time_spent,0,Friday
15,0.304922,0.808577,0.366407,0.865675,0.0,0.0
23,-0.849380,-0.687389,-0.923583,-1.332985,0.0,0.0
30,2.805910,1.012572,2.602791,3.430779,0.0,0.0
31,2.407676,2.576537,2.277279,1.476414,0.0,0.0
38,0.997504,-0.687389,0.703974,0.987823,0.0,0.0


In [53]:
Xtrain_normalized.head()

,trip_distance,tip_amount,total_amount,time_spent,0,Friday
0,-0.945572,1.930551,1.095793,-1.577281,0.0,0.0
1,1.405357,-0.687389,0.975234,1.720710,0.0,0.0
2,-0.233752,-0.134902,-0.237899,0.010641,0.0,0.0
3,-0.743569,-0.687389,-0.863303,-1.210838,0.0,0.0
4,-0.676235,-0.687389,-0.772883,-1.088690,0.0,0.0
